In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'gabor'
CHANNEL = 'gray'
PARAM_CSV = 'gabor_new.csv' # only use for Gabor

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_gabor_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))
param_df = pd.read_csv(os.path.join(ROOT_DIR, "gabor", PARAM_CSV))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters
elif 'gabor' in TRANSFORM:
    GROUPS = param_df['index']

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,119.90,0.0,-0.00,0.00,0.00,-0.00,-0.00,-0.01,0.00,-0.00,...,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00
1,0.00,47.6,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,...,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00
2,-0.00,0.0,21.68,-0.00,0.00,0.00,0.00,0.00,0.00,-0.00,...,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00
3,0.00,-0.0,-0.00,8.81,-0.00,0.00,-0.00,-0.00,-0.00,0.00,...,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00
4,0.00,-0.0,0.00,-0.00,3.56,-0.00,0.00,0.00,-0.00,0.00,...,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00
5,-0.00,-0.0,0.00,0.00,-0.00,1.03,0.00,-0.00,0.00,-0.00,...,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00
6,-0.00,-0.0,0.00,-0.00,0.00,0.00,0.61,0.00,0.00,0.00,...,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00
7,-0.01,0.0,0.00,-0.00,0.00,-0.00,0.00,166.02,-0.00,0.00,...,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00
8,0.00,-0.0,0.00,-0.00,-0.00,0.00,0.00,-0.00,65.96,0.00,...,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,1.00000,0.00005,-0.00001,0.00004,0.00001,-0.00004,-0.00003,-0.00004,0.00002,-0.00002,...,-0.00003,0.00008,-0.00001,0.00002,-0.00000,-0.00003,0.00001,0.00001,0.00001,0.00002
1,0.00005,1.00000,0.00007,-0.00006,-0.00002,-0.00005,-0.00001,0.00004,-0.00001,-0.00003,...,-0.00000,-0.00001,0.00000,0.00003,-0.00002,-0.00001,-0.00004,0.00000,0.00001,0.00003
2,-0.00001,0.00007,1.00000,-0.00001,0.00000,0.00004,0.00004,0.00001,0.00004,-0.00002,...,0.00002,-0.00003,0.00004,-0.00004,0.00005,0.00000,-0.00007,-0.00000,0.00006,-0.00000
3,0.00004,-0.00006,-0.00001,1.00000,-0.00003,0.00003,-0.00001,-0.00004,-0.00003,0.00001,...,-0.00003,-0.00004,0.00001,0.00004,0.00000,0.00003,0.00003,0.00002,-0.00003,-0.00000
4,0.00001,-0.00002,0.00000,-0.00003,1.00000,-0.00002,0.00000,0.00000,-0.00001,0.00006,...,0.00000,0.00002,0.00000,-0.00000,0.00003,-0.00004,0.00004,0.00002,0.00004,-0.00005
5,-0.00004,-0.00005,0.00004,0.00003,-0.00002,1.00000,0.00000,-0.00006,0.00003,-0.00001,...,-0.00002,-0.00003,0.00005,-0.00003,0.00001,-0.00000,0.00004,0.00000,0.00002,-0.00002
6,-0.00003,-0.00001,0.00004,-0.00001,0.00000,0.00000,1.00000,0.00002,0.00000,0.00005,...,0.00004,-0.00001,-0.00002,-0.00001,-0.00001,0.00002,-0.00002,0.00004,-0.00003,0.00001
7,-0.00004,0.00004,0.00001,-0.00004,0.00000,-0.00006,0.00002,1.00000,-0.00000,0.00002,...,0.00000,0.00002,-0.00004,0.00001,-0.00005,-0.00002,-0.00004,0.00004,0.00002,-0.00003
8,0.00002,-0.00001,0.00004,-0.00003,-0.00001,0.00003,0.00000,-0.00000,1.00000,0.00002,...,-0.00003,0.00001,0.00001,0.00002,0.00004,-0.00006,0.00003,0.00003,-0.00001,-0.00002


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

0.03912943769778253

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.58673169e+02 4.71869952e+02 3.55404081e+02 3.43064991e+02
 3.18427706e+02 2.69315671e+02 1.03550952e+02 6.85951059e+01
 6.21395217e+01 5.71930498e+01 5.25888970e+01 4.23807508e+01
 1.79335655e+01 1.35873885e+01 1.28728272e+01 1.11911512e+01
 1.02263133e+01 8.75927039e+00 3.48142508e+00 2.39068098e+00
 2.05917071e+00 1.85056047e+00 1.74784243e+00 1.47777005e+00
 5.20322493e-01 3.61683886e-01 3.17846780e-01 2.94560684e-01
 2.69031652e-01 2.08256689e-01 6.04757069e-02 3.56542932e-02
 3.03948275e-02 2.49187713e-02 2.23412028e-02 1.62370088e-02
 8.75972380e-03 6.85715397e-03 4.53583399e-03 3.34567207e-03
 2.55609424e-03 8.25102695e-29]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,-0.031779,-0.006384,-0.002718,-0.001101,-0.000440,-0.000132,-0.000074,0.996028,-0.009707,-0.003481,...,-0.000408,-0.000116,-0.000040,-0.032759,-0.007002,-0.002960,-0.001163,-0.000444,-0.000140,-0.000050
1,-0.067824,-0.007794,-0.003221,-0.001294,-0.000520,-0.000147,-0.000090,0.056224,-0.012337,-0.004135,...,-0.000476,-0.000132,-0.000044,-0.072846,-0.008519,-0.003467,-0.001352,-0.000537,-0.000168,-0.000055
2,-0.497677,-0.003146,-0.001265,-0.000493,-0.000201,-0.000059,-0.000034,0.010312,-0.005252,-0.001610,...,-0.000178,-0.000054,-0.000017,0.861432,-0.003474,-0.001350,-0.000541,-0.000197,-0.000066,-0.000025
3,0.780513,-0.008101,-0.003232,-0.001266,-0.000513,-0.000154,-0.000089,0.024953,-0.013671,-0.004190,...,-0.000479,-0.000129,-0.000044,0.394854,-0.009005,-0.003512,-0.001349,-0.000517,-0.000164,-0.000056
4,0.341682,-0.016580,-0.006509,-0.002540,-0.001023,-0.000297,-0.000179,0.043134,-0.028537,-0.008398,...,-0.000951,-0.000269,-0.000092,0.278837,-0.018341,-0.007069,-0.002711,-0.001056,-0.000334,-0.000116
5,0.138566,-0.020848,-0.007889,-0.003079,-0.001242,-0.000362,-0.000209,0.038620,-0.037950,-0.010267,...,-0.001161,-0.000333,-0.000106,0.127977,-0.023203,-0.008565,-0.003276,-0.001269,-0.000401,-0.000136
6,0.018655,-0.037968,-0.009037,-0.003348,-0.001329,-0.000381,-0.000226,0.011244,0.993639,-0.012400,...,-0.001241,-0.000352,-0.000114,0.018294,-0.048697,-0.010055,-0.003523,-0.001351,-0.000429,-0.000148
7,0.009318,-0.078999,-0.008346,-0.002891,-0.001169,-0.000334,-0.000195,0.006019,0.038806,-0.012029,...,-0.001083,-0.000294,-0.000102,0.009257,-0.214651,-0.009286,-0.003076,-0.001171,-0.000366,-0.000129
8,0.009395,-0.157080,-0.009600,-0.003295,-0.001295,-0.000378,-0.000222,0.006187,0.033759,-0.014220,...,-0.001216,-0.000338,-0.000117,0.009262,0.927935,-0.010898,-0.003496,-0.001339,-0.000417,-0.000144
9,0.008076,-0.489326,-0.009453,-0.003129,-0.001245,-0.000345,-0.000209,0.005348,0.026653,-0.013983,...,-0.001140,-0.000323,-0.000105,0.007970,0.162597,-0.010572,-0.003328,-0.001266,-0.000402,-0.000139


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine')
cos_dist =  [np.min(row) for row in cos_dist]
cos_dist


[0.003971967967088652,
 0.009176876061217976,
 0.13856763503850178,
 0.21948691269468956,
 0.1457858165823357,
 0.03923414154503668,
 0.006360974975995526,
 0.03428751080812398,
 0.07206499930448218,
 0.1508421980433714,
 0.16398478924118876,
 0.037060439913253806,
 0.013916096307096226,
 0.16477765622048246,
 0.20313301144589502,
 0.12927826491310346,
 0.1907190907077443,
 0.12538151682056264,
 0.014116430696002102,
 0.039681431376249576,
 0.07958196004576579,
 0.14579722181099664,
 0.293117180714331,
 0.3826208479414087,
 0.021879405201795543,
 0.0602388100868233,
 0.17873352373363316,
 0.27845968402859334,
 0.28805576982074943,
 0.4236465092007018,
 0.025811605502281787,
 0.0871277703214558,
 0.13935677449168793,
 0.17179047587212437,
 0.28403220670250307,
 0.429446591295776,
 0.23752297805511002,
 0.35801851805515617,
 0.22720543143601835,
 0.31978861900991873,
 0.1931064213468442,
 0.494298625739353]